## 載入模型

In [297]:
label_to_coordinates = {
    "1-1": (0, 0), "1-2": (0.6, 0), "1-3": (1.2, 0), "1-4": (1.8, 0), "1-5": (2.4, 0), "1-6": (3.0, 0),"1-7": (3.6, 0), "1-8": (4.2, 0), "1-9": (4.8, 0), "1-10": (5.4, 0), "1-11": (6.0, 0),
    "2-1": (0, 0.6), "2-11": (6.0, 0.6),
    "3-1": (0, 1.2), "3-11": (6.0, 1.2),
    "4-1": (0, 1.8), "4-11": (6.0, 1.8),
    "5-1": (0, 2.4), "5-11": (6.0, 2.4),
    "6-1": (0, 3.0), "6-2": (0.6, 3.0), "6-3": (1.2, 3.0), "6-4": (1.8, 3.0), "6-5": (2.4, 3.0),"6-6": (3.0, 3.0), "6-7": (3.6, 3.0), "6-8": (4.2, 3.0), "6-9": (4.8, 3.0), "6-10": (5.4, 3.0), "6-11": (6.0, 3.0),
    "7-1": (0, 3.6), "7-11": (6.0, 3.6),
    "8-1": (0, 4.2), "8-11": (6.0, 4.2),
    "9-1": (0, 4.8), "9-11": (6.0, 4.8),
    "10-1": (0, 5.4), "10-11": (6.0, 5.4),
    "11-1": (0, 6.0), "11-2": (0.6, 6.0), "11-3": (1.2, 6.0), "11-4": (1.8, 6.0), "11-5": (2.4, 6.0),"11-6": (3.0, 6.0), "11-7": (3.6, 6.0), "11-8": (4.2, 6.0), "11-9": (4.8, 6.0), "11-10": (5.4, 6.0), "11-11": (6.0, 6.0)
}
label_mapping = {
    '11': '1-1','10': '1-2','9': '1-3','8': '1-4','7': '1-5','6': '1-6','5': '1-7','4': '1-8','3': '1-9','2': '1-10','1': '1-11',
    '12': '2-1','30': '2-11',
    '13': '3-1','29': '3-11',
    '14': '4-1','28': '4-11',
    '15': '5-1','27': '5-11',
    '16': '6-1','17': '6-2','18': '6-3','19': '6-4','20': '6-5','21': '6-6','22': '6-7','23': '6-8','24': '6-9','25': '6-10','26': '6-11',
    '49': '7-1','31': '7-11',
    '48': '8-1','32': '8-11',
    '47': '9-1','33': '9-11',
    '46': '10-1','34': '10-11',
    '45': '11-1','44': '11-2','43': '11-3','42': '11-4','41': '11-5','40': '11-6','39': '11-7','38': '11-8','37': '11-9','36': '11-10','35': '11-11'
}

In [298]:
from tensorflow.keras.models import load_model
from tensorflow import keras
import pandas as pd
import numpy as np
import tensorflow as tf
import joblib  # 用於保存模型
import json
from sklearn.model_selection import train_test_split

In [ ]:
# 載入模型與標準化器
# base_model = load_model('Model_SAVE/4 mc AP RSSI FTM StdDev 2024-12-14 data train/DNN_best_model4mcAP.h5')
base_model = load_model('transfer model/DNN_best_model4mcAP_1week_20dataperRP.h5')

scaler = joblib.load('Model_SAVE/4 mc AP RSSI FTM StdDev 2024-12-14 data train/scaler_4mcAP.pkl')

In [300]:
# 凍結所有層
for layer in base_model.layers[:-1]:  # 除了最後一層 (Output Layer)
    layer.trainable = False

# 確認哪些層可訓練
base_model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_44 (Dense)            (None, 64)                832       
                                                                 
 dense_45 (Dense)            (None, 128)               8320      
                                                                 
 dense_46 (Dense)            (None, 64)                8256      
                                                                 
 dense_47 (Dense)            (None, 49)                3185      
                                                                 
Total params: 20,593
Trainable params: 3,185
Non-trainable params: 17,408
_________________________________________________________________


In [301]:
base_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### 加入新資料 & 資料前處理

In [302]:
selected_columns = ['Label',
                    'AP1_Distance (mm)','AP2_Distance (mm)','AP3_Distance (mm)','AP4_Distance (mm)',
'AP1_StdDev (mm)','AP2_StdDev (mm)','AP3_StdDev (mm)','AP4_StdDev (mm)',
                                'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi']  

X_testing_selected_columns = [
        'AP1_Distance (mm)','AP2_Distance (mm)','AP3_Distance (mm)','AP4_Distance (mm)',
        'AP1_StdDev (mm)','AP2_StdDev (mm)','AP3_StdDev (mm)','AP4_StdDev (mm)',
                                'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi']  

label_column = 'Label'
target_column = 'Label'
#  'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi'
# 'AP1_Distance (mm)','AP2_Distance (mm)','AP3_Distance (mm)','AP4_Distance (mm)',
# 'AP1_StdDev (mm)','AP2_StdDev (mm)','AP3_StdDev (mm)','AP4_StdDev (mm)',

In [ ]:
# 讀取測試資料
test_file_path = "timestamp_allignment_Balanced_2024_12_27_rtt_logs.csv"  # 測試資料的檔案名稱
date_test = "2024_12_27"
modelname = "DNN 4 mcAPs BEST"
test_data = pd.read_csv(test_file_path, usecols=selected_columns)
test_data

,Label,AP1_Distance (mm),AP2_Distance (mm),AP3_Distance (mm),AP4_Distance (mm),AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi,AP1_StdDev (mm),AP2_StdDev (mm),AP3_StdDev (mm),AP4_StdDev (mm)
0,1-1,3999.0,7339.0,14219.0,-303.0,-65.0,-68.0,-65.0,-50.0,986.0,295.0,580.0,968.0
1,1-1,3921.0,7368.0,12835.0,-420.0,-65.0,-68.0,-64.0,-54.0,1071.0,1013.0,765.0,671.0
2,1-1,4038.0,6928.0,12510.0,-651.0,-65.0,-69.0,-64.0,-52.0,986.0,375.0,562.0,770.0
3,1-1,4038.0,7163.0,13955.0,-381.0,-65.0,-68.0,-65.0,-54.0,1042.0,267.0,527.0,555.0
4,1-1,4077.0,7163.0,13281.0,-342.0,-65.0,-68.0,-64.0,-54.0,996.0,137.0,826.0,428.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20673,9-11,1694.0,5083.0,2666.0,5438.0,-55.0,-67.0,-50.0,-66.0,294.0,1093.0,369.0,2384.0
20674,9-11,NaN,5229.0,NaN,NaN,NaN,-65.0,NaN,NaN,NaN,1125.0,NaN,NaN
20675,9-11,2007.0,5346.0,2627.0,4696.0,-55.0,-66.0,-50.0,-66.0,152.0,1073.0,482.0,463.0
20676,9-11,1929.0,5434.0,2549.0,4735.0,-55.0,-65.0,-51.0,-66.0,216.0,964.0,568.0,481.0


In [304]:
# 資料前處理 (一): 刪除前後n筆資料
n = 10
# 確保依據Label排序
test_data = test_data.sort_values(by=label_column).reset_index(drop=True)

# 建立一個空的 DataFrame 用於存放處理後的資料
test_processed_data = pd.DataFrame(columns=test_data.columns)

# 針對每個Label群組進行處理
for label, group in test_data.groupby(label_column):
    # 刪除前n筆和後n筆資料
    if len(group) > 2 * n:  # 確保群組資料足夠
        group = group.iloc[n:-n]
    else:
        group = pd.DataFrame()  # 若資料不足，刪除整個群組
    # 將處理後的群組資料加入
    test_processed_data = pd.concat([test_processed_data, group], ignore_index=True)

# test_processed_data
# Calculate the number of rows with NaN values
nan_rows = test_processed_data.isnull().any(axis=1).sum()

# Print the result
print(f"Number of rows with NaN values: {nan_rows}")

# 找出包含 NaN 的列
rows_with_nan = test_processed_data[test_processed_data.isnull().any(axis=1)]

# # 印出這些列
# print("Rows with NaN values:")
# print(rows_with_nan)
test_data_imputed = test_processed_data.groupby(label_column).apply(
    lambda group: group.fillna(group.mean())
).reset_index()

# Calculate the number of rows with NaN values
nan_rows = test_data_imputed.isnull().any(axis=1).sum()

# Print the result
print(f"Number of rows with NaN values: {nan_rows}")

# 找出包含 NaN 的列
rows_with_nan = test_data_imputed[test_data_imputed.isnull().any(axis=1)]

test_data_imputed

Number of rows with NaN values: 974
Number of rows with NaN values: 0


,Label,level_1,AP1_Distance (mm),AP2_Distance (mm),AP3_Distance (mm),AP4_Distance (mm),AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi,AP1_StdDev (mm),AP2_StdDev (mm),AP3_StdDev (mm),AP4_StdDev (mm)
0,1-1,0,4038.0,7358.0,13018.0,-1855.0,-65.0,-68.0,-64.0,-50.0,965.0,877.0,314.0,2003.0
1,1-1,1,3960.0,7241.0,13252.0,-342.0,-65.0,-68.0,-66.0,-54.0,1056.0,880.0,160.0,483.0
2,1-1,2,3843.0,7202.0,13838.0,-303.0,-65.0,-68.0,-64.0,-54.0,1053.0,876.0,714.0,583.0
3,1-1,3,3960.0,7397.0,14307.0,-342.0,-65.0,-68.0,-64.0,-54.0,938.0,105.0,265.0,522.0
4,1-1,4,3999.0,7075.0,13174.0,-460.0,-65.0,-68.0,-66.0,-54.0,959.0,303.0,341.0,637.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19693,9-11,19693,1850.0,5464.0,2666.0,4852.0,-55.0,-65.0,-50.0,-67.0,215.0,959.0,409.0,497.0
19694,9-11,19694,1967.0,5112.0,2353.0,4881.0,-55.0,-66.0,-50.0,-67.0,141.0,1158.0,709.0,577.0
19695,9-11,19695,1968.0,5024.0,2366.0,4705.0,-55.0,-67.0,-48.0,-66.0,175.0,1166.0,671.0,491.0
19696,9-11,19696,1772.0,5434.0,2666.0,5008.0,-55.0,-66.0,-52.0,-67.0,151.0,861.0,519.0,483.0


In [305]:
reverse_label_mapping = {v: int(k) - 1 for k, v in label_mapping.items()}  # 讓數字標籤 -1

In [306]:
# 建立 Label 映射
y_test = test_data_imputed[target_column]
y_test_numeric = y_test.map(reverse_label_mapping)

print("Final reverse_label_mapping in DNN:", reverse_label_mapping)
print("y_numeric unique values in DNN:", y_test_numeric.unique())

y_test_numeric

Final reverse_label_mapping in DNN: {'1-1': 10, '1-2': 9, '1-3': 8, '1-4': 7, '1-5': 6, '1-6': 5, '1-7': 4, '1-8': 3, '1-9': 2, '1-10': 1, '1-11': 0, '2-1': 11, '2-11': 29, '3-1': 12, '3-11': 28, '4-1': 13, '4-11': 27, '5-1': 14, '5-11': 26, '6-1': 15, '6-2': 16, '6-3': 17, '6-4': 18, '6-5': 19, '6-6': 20, '6-7': 21, '6-8': 22, '6-9': 23, '6-10': 24, '6-11': 25, '7-1': 48, '7-11': 30, '8-1': 47, '8-11': 31, '9-1': 46, '9-11': 32, '10-1': 45, '10-11': 33, '11-1': 44, '11-2': 43, '11-3': 42, '11-4': 41, '11-5': 40, '11-6': 39, '11-7': 38, '11-8': 37, '11-9': 36, '11-10': 35, '11-11': 34}
y_numeric unique values in DNN: [10  1  0  9  8  7  6  5  4  3  2 45 33 44 35 34 43 42 41 40 39 38 37 36
 11 29 12 28 13 27 14 26 15 24 25 16 17 18 19 20 21 22 23 48 30 47 31 46
 32]


0        10
1        10
2        10
3        10
4        10
         ..
19693    32
19694    32
19695    32
19696    32
19697    32
Name: Label, Length: 19698, dtype: int64

In [307]:
# 把label部分拿掉
X_test = test_data_imputed.drop(columns=['level_1','Label'])

# 確保測試資料的特徵與訓練資料的特徵一致
X_test = X_test[X_testing_selected_columns]  # 選取相同的特徵

print(type(X_test))

<class 'pandas.core.frame.DataFrame'>


In [308]:
# 使用之前訓練時的標準化器 (scaler) 來標準化測試數據
X_test_scaled = scaler.transform(X_test)

### 重新訓練model

In [ ]:
# 會發現如果用 train_test_split的方法會有資料分布不平均問題，解決辦法如下
from sklearn.model_selection import StratifiedShuffleSplit

# 設定每個類別的固定樣本數
N_train = 20 # 訓練集每個類別至少要有 N_train 筆資料
test_val_ratio = 0.5  # 剩餘資料中，50% 作為驗證集，50% 作為測試集

# 轉為 DataFrame 方便操作
data = pd.DataFrame(X_test_scaled)
data['label'] = y_test_numeric  # 加入 label 欄位

print((data['label'] == 10).sum())  # 直接計算 True 的數量


402


In [310]:
data

,0,1,2,3,4,5,6,7,8,9,10,11,label
0,0.533468,0.727178,2.342175,-1.876067,0.086528,0.183418,-0.767232,2.021751,-0.973450,-0.144781,-1.105064,1.190134,10
1,0.500544,0.687173,2.431013,-1.250919,0.232493,0.188003,-1.072440,-0.747345,-0.973450,-0.144781,-1.484683,0.570201,10
2,0.451158,0.673838,2.653488,-1.234804,0.227681,0.181890,0.025515,-0.565168,-0.973450,-0.144781,-1.105064,0.570201,10
3,0.500544,0.740513,2.831544,-1.250919,0.043220,-0.996340,-0.864344,-0.676296,-0.973450,-0.144781,-1.105064,0.570201,10
4,0.517006,0.630414,2.401401,-1.299675,0.076904,-0.693759,-0.713722,-0.466792,-0.973450,-0.144781,-1.484683,0.570201,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19693,-0.390092,0.079578,-1.587958,0.895163,-1.116482,0.308729,-0.578955,-0.721841,1.158218,0.403943,1.552270,-1.444580,32
19694,-0.340706,-0.040779,-1.706788,0.907146,-1.235179,0.612838,0.015606,-0.576099,1.158218,0.221035,1.552270,-1.444580,32
19695,-0.340284,-0.070868,-1.701852,0.834425,-1.180643,0.625063,-0.059705,-0.732771,1.158218,0.038127,1.931890,-1.289597,32
19696,-0.423016,0.069320,-1.587958,0.959620,-1.219139,0.158967,-0.360949,-0.747345,1.158218,0.221035,1.172651,-1.444580,32


In [311]:
# 儲存訓練集
train_data = data.groupby('label', group_keys=False).sample(n=N_train, replace=False, random_state=42)
# print(train_data.index)  # 查看 train_data 的索引

print((train_data['label'] == 10).sum())  # 計算 label 為 10 的數量


10


In [312]:
remaining_data = data.drop(train_data.index)
print(remaining_data['label'].value_counts().get(10, 0))  # 應該是 348


392


In [313]:
# 取得剩餘資料的 features 和 labels
X_remaining = remaining_data.drop(columns=['label'])
y_remaining = remaining_data['label']

print(y_remaining.value_counts().get(10, 0))

392


In [314]:
# 使用 StratifiedShuffleSplit 來確保驗證集與測試集的類別比例一致
sss = StratifiedShuffleSplit(n_splits=1, test_size=test_val_ratio, random_state=42)
for val_index, test_index in sss.split(X_remaining, y_remaining):
    X_val, X_test = X_remaining.iloc[val_index], X_remaining.iloc[test_index]
    y_val, y_test = y_remaining.iloc[val_index], y_remaining.iloc[test_index]

# 轉換回 NumPy 陣列
X_train_small, y_train_small = train_data.drop(columns=['label']).values, train_data['label'].values
X_val, y_val = X_val.values, y_val.values
X_test, y_test = X_test.values, y_test.values

# 確認切分後的數據數量
print(f"Training set: {len(X_train_small)} samples, {len(np.unique(y_train_small))} unique labels")
print(f"Validation set: {len(X_val)} samples, {len(np.unique(y_val))} unique labels")
print(f"Test set: {len(X_test)} samples, {len(np.unique(y_test))} unique labels")

Training set: 490 samples, 49 unique labels
Validation set: 9604 samples, 49 unique labels
Test set: 9604 samples, 49 unique labels


In [315]:
# # 取出 10% 資料作為訓練資料，剩下的 90% 作為測試資料
# X_train_small, X_unused, y_train_small, y_unused = train_test_split(
#     X_test_scaled, y_test_numeric, test_size=0.99, random_state=42, stratify=y_test_numeric
# )
# # 再切出一半作為validation set and test set
# X_val, X_test, y_val, y_test = train_test_split(
#     X_unused, y_unused, test_size=0.5, random_state=42, stratify=y_unused
# )

In [316]:
# 檢查各組資料比例
print(f"Training set: {len(X_train_small)} samples")
print(f"Validation set: {len(X_val)} samples")
print(f"Test set: {len(X_test)} samples")

Training set: 490 samples
Validation set: 9604 samples
Test set: 9604 samples


In [317]:
import pandas as pd

# 計算每個 Set 內各 Label 的資料數量
train_label_counts = pd.Series(y_train_small).value_counts().sort_index()
val_label_counts = pd.Series(y_val).value_counts().sort_index()
test_label_counts = pd.Series(y_test).value_counts().sort_index()

# # 印出統計結果
# print("Training Set - Label Counts:")
# print(train_label_counts)

# print("\nValidation Set - Label Counts:")
# print(val_label_counts)

# print("\nTest Set - Label Counts:")
# print(test_label_counts)

# 確保所有 Labels 都有出現在三個 Set 裡
all_labels = sorted(set(train_label_counts.index) | set(val_label_counts.index) | set(test_label_counts.index))
label_distribution = pd.DataFrame(index=all_labels)

label_distribution["Training Set"] = train_label_counts
label_distribution["Validation Set"] = val_label_counts
label_distribution["Test Set"] = test_label_counts

# 用 0 填補缺失值（表示該 Label 在該 Set 中沒有數據）
label_distribution = label_distribution.fillna(0).astype(int)

from IPython.display import display
display(label_distribution)


,Training Set,Validation Set,Test Set
0,10,196,196
1,10,196,196
2,10,196,196
3,10,196,196
4,10,196,196
5,10,196,196
6,10,196,196
7,10,196,196
8,10,196,196
9,10,196,196


In [318]:
import time
# 記錄開始時間
start_time = time.time()

In [319]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [320]:
base_model.fit(X_train_small, y_train_small,validation_data=(X_val, y_val), epochs=10000, batch_size=32, callbacks=[early_stop])

Epoch 1/10000
16/16 [==============================] - 0s 15ms/step - loss: 5.2216 - accuracy: 0.4388 - val_loss: 4.2195 - val_accuracy: 0.4767
Epoch 2/10000
16/16 [==============================] - 0s 8ms/step - loss: 4.3590 - accuracy: 0.4959 - val_loss: 3.5654 - val_accuracy: 0.5249
Epoch 3/10000
16/16 [==============================] - 0s 6ms/step - loss: 3.7908 - accuracy: 0.5429 - val_loss: 3.1375 - val_accuracy: 0.5646
Epoch 4/10000
16/16 [==============================] - 0s 11ms/step - loss: 3.3888 - accuracy: 0.5776 - val_loss: 2.8073 - val_accuracy: 0.5928
Epoch 5/10000
16/16 [==============================] - 0s 11ms/step - loss: 3.0808 - accuracy: 0.6122 - val_loss: 2.5611 - val_accuracy: 0.6166
Epoch 6/10000
16/16 [==============================] - 0s 12ms/step - loss: 2.8194 - accuracy: 0.6224 - val_loss: 2.3583 - val_accuracy: 0.6344
Epoch 7/10000
16/16 [==============================] - 0s 7ms/step - loss: 2.5952 - accuracy: 0.6347 - val_loss: 2.1691 - val_accuracy: 0.

In [321]:
# 記錄結束時間
end_time = time.time()

# 計算訓練時間（秒）
training_time = end_time - start_time
print(f"訓練時間：{training_time:.2f} 秒")

訓練時間：60.53 秒


In [322]:
# 使用剩餘的 90% 測試資料進行模型評估
loss, accuracy = base_model.evaluate(X_test, y_test)
print(f"Evaluation on 90% unused data - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

  1/301 [..............................] - ETA: 2s - loss: 0.1109 - accuracy: 0.9375

301/301 [==============================] - 0s 453us/step - loss: 0.3812 - accuracy: 0.9592
Evaluation on 90% unused data - Loss: 0.3812, Accuracy: 0.9592


In [323]:
# ### 預測
# base_model.evaluate(X_test_scaled, y_test_numeric)  # 確保模型在測試模式

In [324]:
# 預測測試資料
y_test_pred_numeric = base_model.predict(X_test_scaled)
y_pred_classes = np.argmax(y_test_pred_numeric, axis=1)

# 轉換為原本的 Label
y_test_pred_labels = [label_mapping[str(num + 1)] for num in y_pred_classes]  # 補回 +1
y_test_pred_labels

616/616 [==============================] - 0s 460us/step


['1-3',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-3',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-3',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-3',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',
 '1-1',


In [325]:
# 讀取測試資料的實際 Label
y_test_actual = test_data_imputed[target_column]
test_data_imputed

,Label,level_1,AP1_Distance (mm),AP2_Distance (mm),AP3_Distance (mm),AP4_Distance (mm),AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi,AP1_StdDev (mm),AP2_StdDev (mm),AP3_StdDev (mm),AP4_StdDev (mm)
0,1-1,0,4038.0,7358.0,13018.0,-1855.0,-65.0,-68.0,-64.0,-50.0,965.0,877.0,314.0,2003.0
1,1-1,1,3960.0,7241.0,13252.0,-342.0,-65.0,-68.0,-66.0,-54.0,1056.0,880.0,160.0,483.0
2,1-1,2,3843.0,7202.0,13838.0,-303.0,-65.0,-68.0,-64.0,-54.0,1053.0,876.0,714.0,583.0
3,1-1,3,3960.0,7397.0,14307.0,-342.0,-65.0,-68.0,-64.0,-54.0,938.0,105.0,265.0,522.0
4,1-1,4,3999.0,7075.0,13174.0,-460.0,-65.0,-68.0,-66.0,-54.0,959.0,303.0,341.0,637.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19693,9-11,19693,1850.0,5464.0,2666.0,4852.0,-55.0,-65.0,-50.0,-67.0,215.0,959.0,409.0,497.0
19694,9-11,19694,1967.0,5112.0,2353.0,4881.0,-55.0,-66.0,-50.0,-67.0,141.0,1158.0,709.0,577.0
19695,9-11,19695,1968.0,5024.0,2366.0,4705.0,-55.0,-67.0,-48.0,-66.0,175.0,1166.0,671.0,491.0
19696,9-11,19696,1772.0,5434.0,2666.0,5008.0,-55.0,-66.0,-52.0,-67.0,151.0,861.0,519.0,483.0


In [326]:
# 取得預測與實際座標
y_test_pred_coordinates = np.array([label_to_coordinates[label] for label in y_test_pred_labels])
y_test_actual_coordinates = np.array([label_to_coordinates[label] for label in y_test_actual])

# 計算 MDE (Mean Distance Error)
distances = np.linalg.norm(y_test_pred_coordinates - y_test_actual_coordinates, axis=1)
mean_mde = np.mean(distances)

# 記錄每個 RP 的 MDE
mde_report_test = {}
for true_label, distance in zip(y_test_actual, distances):
    if true_label not in mde_report_test:
        mde_report_test[true_label] = []
    mde_report_test[true_label].append(distance)

# 計算測試資料的 MDE 平均值
mde_report_test_avg = {label: {"mde": np.mean(dists), "count": len(dists)} for label, dists in mde_report_test.items()}

# 儲存 MDE 結果到 JSON 檔案
test_file_path = f"{modelname}_{date_test}.json"
with open(test_file_path, "w") as f:
    json.dump(mde_report_test_avg, f, indent=4)

print(f"Test Data MDE report saved to: {test_file_path}")
print(f"\nTest Data Mean MDE: {mean_mde:.4f} meters")


Test Data MDE report saved to: DNN 4 mcAPs BEST_2025_01_10.json

Test Data Mean MDE: 0.0668 meters


In [ ]:
base_model.save(f"DNN_best_model4mcAP_1week_to_2week_20dataPerRP.h5")